# Data Preparation Notebook
Author: Andrew Auyeung  
This notebook is used to load the LDA model trained on google colab and export the 
document-topic-matrix as a CSV for visualizations.

In [1]:
# load libraries
from pyspark.ml import PipelineModel
import pyspark

In [2]:
# Start Spark
spark = pyspark.sql.SparkSession.builder.config("spark.driver.memory", "15g").getOrCreate()
spark.getActiveSession()

In [3]:
# Load Model
lda_model  = PipelineModel.load('../models/lda_article_model')

# Load CSV into spark
articles = spark.read.csv('../src/TDS_articles.csv', inferSchema=True, header=True, sep="\t")

In [4]:
def show_topics(vocab, topic_indexes, topic_labels=None):
    """
    Helper function to show topics from the pyspark LDA results
    args:
        vocab (list): list of all words in the vectorized vocabulary
        topic_indexes (list): indexes of words associated with topics from LDA
        topic_labels (list): labeled topic names.  Naively matches topics with 
            number of topics. i.e. topic 1, topic 2... topic n
    """
    if not topic_labels:
        topic_labels = [f"Topic: {str(i)}" for i in range(len(topic_indexes))]
        assert len(topic_labels) == len(topic_indexes)

    for label, words in zip(topic_labels, topic_indexes):
        topic_words = ", ".join([vocab[word_idx] for word_idx in words])
        print(label + " " + topic_words)

In [6]:
vocab = lda_model.stages[2].vocabulary
topics = lda_model.stages[4].describeTopics().collect()
topic_idx = [topic.termIndices for topic in topics]

In [7]:
show_topics(vocab, topic_idx)

Topic: 0 ai, customer, rating, marketing, city, human, technology, min, intelligence, patient
Topic: 1 country, covid, loan, coronavirus, death, county, population, virus, gdp, rate
Topic: 2 table, join, query, column, sql, dataframe, price, database, row, yard
Topic: 3 inceptionv, yolov, deeplabv, fm, rcnn, resnet, multipathnet, stn, cvpr, resnext
Topic: 4 image, file, model, function, layer, training, code, network, python, action
Topic: 5 scraping, scrapy, selenium, espresso, scrape, soup, spider, html, web, page
Topic: 6 player, game, season, team, league, players, nba, win, shot, football
Topic: 7 song, music, automl, rasa, aws, spotify, artist, songs, iris, lyric
Topic: 8 wine, minority, chromosome, imbalanced, smote, rfm, oversampling, ros, radar, customer
Topic: 9 quantum, airflow, exam, election, vote, voter, centrality, dag, qubits, bigquery
Topic: 10 word, text, sentence, document, vector, language, sentiment, nlp, sequence, embedding
Topic: 11 agent, infected, arxiv, al, et

In [5]:
# Set up cleaning to prep articles:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

import re

lemmatizer = WordNetLemmatizer()  
def remove_digits(text):
    """
    Uses Regex to replace anything that is not an alphabetic character with an empty string
    """
    return re.sub(r'[^A-Za-z\s]', ' ', text)

def lemma(text, pos_to_avoid=[]):
    """
    Lemmatize Text.  Pass in a list of POS to avoid when lemmatizing
    """
    text = [lemmatizer.lemmatize(token) for token in text.split()]
    return ' '.join(text)

def remove_emojis(text):
    """
    Remove emojis from text
    """
    return re.sub(r':\w*:', ' ', text)
def remove_multiple_spaces(text):
    """
    Remove multiple consecutive spaces from text
    """
    text = re.sub(r'\s+', ' ', text.strip())
    return text

def clean_doc(text):

    # remove emojis
    text = remove_emojis(text)
    # lemmatize and remove pronouns
    text = lemma(text, ['-PRON-'])
    # remove digits and punctuation
    text = remove_digits(text)
    # change all to lowercase
    text = text.lower()
    # remove blank spaces
    text = remove_multiple_spaces(text)
    
    return text

[nltk_data] Downloading package wordnet to /Users/andrew/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andrew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Register cleaning function as UDF 
from pyspark.sql.functions import udf, split, explode, col, posexplode
from pyspark.sql.types import *

# add the cleaning function as a UDF
clean_udf = udf(clean_doc)

# make a UDF to remove the bracket delimiters
remove_brackets = udf(lambda row: row.replace('}"', '').replace('"{"', ''))

In [7]:
# apply remove brackets to body column
articles = (articles
  .withColumn('body', remove_brackets('body'))
  )

articles.show(5)

+----------+--------------------+--------------------+--------------------+----------+-----+------+----------+--------------------+--------------------+
|article_id|               title|            subtitle|              author|      date|claps|images|codeblocks|                link|                body|
+----------+--------------------+--------------------+--------------------+----------+-----+------+----------+--------------------+--------------------+
|     42343|Mastering Data Ag...|Aggregating Data ...|Sadrach Pierre, P...|2020-09-03|   40|     8|         8|https://towardsda...|"Data aggregation...|
|     42356|1 trick to find a...|Search and Contri...|        Satyam Kumar|2020-09-04|    7|     4|         0|https://towardsda...|"There are thousa...|
|     42424|How to Extract Da...|Explore the power...|     Yong Cui, Ph.D.|2020-09-05|  115|     1|         0|https://towardsda...|"One key operatio...|
|     42431|Delta Lake in Act...|Beginner’s Guide ...|        Jyoti Dhiman|2020-09

In [9]:
train_corpa = (
    articles
        .select('article_id', 'date', 'body')
        .where(col('body').isNotNull())
        .withColumn('body', clean_udf('body'))
        .withColumnRenamed('body', 'text')
    )

train_corpa.show(5)

+----------+----------+--------------------+
|article_id|      date|                text|
+----------+----------+--------------------+
|     42343|2020-09-03|data aggregation ...|
|     42356|2020-09-04|there are thousan...|
|     42424|2020-09-05|one key operation...|
|     42431|2020-09-05|this is a follow ...|
|     42445|2020-09-06|structured query ...|
+----------+----------+--------------------+
only showing top 5 rows



In [38]:
text = articles.select('body').collect()

In [39]:
text = pd.DataFrame(text, columns=['body'])

In [46]:
text['body'] = text['body'].map(lambda x: " ".join(str(x).split('","'))).map(lambda x: x[1:-2])

In [47]:
text.to_csv('../src/text.csv')

In [10]:
clean_text = train_corpa.collect()

In [18]:
import pandas as pd
clean_text = pd.DataFrame(clean_text, columns=['article_id', 'date', 'text'])

In [52]:
final_n = pd.read_csv("../src/TDS_document_topic_matrix_cleaned.csv", sep='\t')

In [57]:
len(final_n['article_id'])

35623

In [62]:
clean_text[clean_text['article_id'].isin(final_n['article_id'])].to_csv('../src/clean_text.csv', sep='\t')

In [20]:
clean_text.to_csv('../src/clean_text.csv', sep='\t')

In [12]:
doc_topic_mat = lda_model.transform(train_corpa)
doc_topic_mat.show(2)

+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|article_id|      date|                text|              tokens|     filtered_tokens|           term_freq|            features|   topicDistribution|
+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      3406|2018-03-24|there is a theore...|[there, is, a, th...|[theorem, telling...|(262144,[0,1,2,4,...|(262144,[0,1,2,4,...|[6.17526639750102...|
|      5405|2019-07-02|this article is a...|[this, article, i...|[article, case, k...|(262144,[0,1,2,4,...|(262144,[0,1,2,4,...|[0.09939988428045...|
+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [16]:
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import round
# Save topic matrix as csv to export to tableau
dtm = (doc_topic_mat
    .withColumn('distribution_array', vector_to_array(doc_topic_mat['topicDistribution']))
    .select(
        ['article_id', 'date', 'text'] + 
        [round(col('distribution_array')[i], scale=3).alias(f"Topic {str(i)}") for i in range(20)])
    )

dtm.show()

+----------+----------+--------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|article_id|      date|                text|Topic 0|Topic 1|Topic 2|Topic 3|Topic 4|Topic 5|Topic 6|Topic 7|Topic 8|Topic 9|Topic 10|Topic 11|Topic 12|Topic 13|Topic 14|Topic 15|Topic 16|Topic 17|Topic 18|Topic 19|
+----------+----------+--------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|      3406|2018-03-24|there is a theore...|    0.0|    0.0|    0.0|    0.0|  0.162|    0.0|    0.0|    0.0|    0.0|    0.0|     0.0|     0.0|   0.191|     0.0|   0.629|     0.0|     0.0|     0.0|     0.0|   0.017|
|      5405|2019-07-02|this article is a...|  0.099|    0.0|  0.017|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|     0.0|     0.

In [18]:
import pandas as pd

In [19]:
dtm_df = dtm.toPandas()

In [20]:
dtm_df.head()

,article_id,date,text,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,...,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19
0,3406,2018-03-24,there is a theorem telling u there is no singl...,0.000,0.0,0.000,0.0,0.162,0.000,0.0,...,0.000,0.000,0.191,0.000,0.629,0.0,0.0,0.000,0.0,0.017
1,5405,2019-07-02,this article is a case for keeping retaining a...,0.099,0.0,0.017,0.0,0.000,0.000,0.0,...,0.000,0.000,0.852,0.015,0.000,0.0,0.0,0.016,0.0,0.000
2,5957,2019-11-26,why are competent people often so bad at teach...,0.153,0.0,0.000,0.0,0.000,0.000,0.0,...,0.000,0.045,0.802,0.000,0.000,0.0,0.0,0.000,0.0,0.000
3,6932,None,this article wa originally posted by derrick m...,0.000,0.0,0.000,0.0,0.256,0.016,0.0,...,0.554,0.000,0.000,0.000,0.173,0.0,0.0,0.000,0.0,0.000
4,8041,2019-07-20,in a previous article we discussed the concept...,0.000,0.0,0.000,0.0,0.000,0.000,0.0,...,0.000,0.000,0.065,0.000,0.935,0.0,0.0,0.000,0.0,0.000


In [21]:
dtm_df.shape

(35646, 23)

In [22]:
dtm_df.to_csv('../src/TDS_document_topic_matrix.csv', sep="\t")